<a href="https://colab.research.google.com/github/sakethbachu/UNET/blob/master/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

     |████████████████████████████████| 592.3MB 74.6MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.60 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [0]:
!pip3 install torchvision

     |████████████████████████████████| 734.6MB 12kB/s 
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
import os

In [0]:

from google.colab import drive
drive.mount('/content/gdrive');

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Defining two functions, one loading the raw input images and the other loading the labels. Meanwhile resizing the images to 128*128. Converting the labels to gray scale images as CE loss requires (N).N:mini batch size. The softmax is already applied for the CEloss.

In [0]:

def load_img(folder):
  #an iter variable
    images = []
    for filename in os.listdir(folder): 
        img = cv2.imread(os.path.join(folder, filename)) 
        img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
        images.append(img)
    return images

def load_label_img(folder):
  img1 = [] #an empty list for the files
  for filename in os.listdir(folder): 
      img = cv2.imread(os.path.join(folder, filename))
      img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      img1.append(img)
  return img1

Converting the colors to grayscale and appending them in a list. This helps in giving back the index which will be the class number and this labels the image. The new labels will be in the form of numpy array of the form (128, 128, 1).

In [0]:
def colortogray(cn):
    cn = np.reshape(cn, (1, 1, 3));
    cn = cv2.cvtColor(cn, cv2.COLOR_BGR2GRAY);
    return cn
#these are the colors that are used for making the boundaries(ie classfication colours)    
colors = [];
colors.append(colortogray(np.array([64, 128, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 0, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 128, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 128, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 0, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 0, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 0, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 128, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 192, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 64, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 0, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 0, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 128, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 0, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 64, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 192, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 64, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 64, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 128, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 0, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 128, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 128, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 128, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 0, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 64, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([128, 64, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 128, 128], dtype = 'uint8')))
colors.append(colortogray(np.array([192, 128, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([64, 0, 64], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 192, 192], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 0, 0], dtype = 'uint8')))
colors.append(colortogray(np.array([0, 192, 64], dtype = 'uint8')))

def class_pixel(label_img):
     #      if label_img[0].any() == c[0] and label_img[1].any() == c[1] and label_img[2].any() == c[2]:
      #for i in range(128):
          #for j in range(128):
            #for k in range(3):
             # if label_img[k, i, j].any() == c.any():
                  #class_pix = index
    #return class_pix

    class_pix = np.ones([128, 128, 1], dtype = int);
    for index, c in enumerate(colors):
        class_pix[label_img == c] = index
    return class_pix

# Convert all segmented images into labeled images with appropriate class number.

def label_img_list(img_list):
    images = []
    for image in img_list:
        images.append(class_pixel(image))
    return images
           

We are implementing the advanced dataloader to implement mini batch size. We are implementing magic methods such as getitem() and len(). The former returns  an item of the given index and the latter gives the length of the dataset. Also we are defining the transforms that are applied for the data that is loaded.

In [0]:
transform_img = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
transform_img_label = transforms.Compose([transforms.ToTensor()])

from torch.utils.data import Dataset, DataLoader

#Advance Dataloaders

class trainset(data.Dataset):
    def __init__(self, transform = None, root_train = None, root_train_label = None, transform_label = None):
        self.train_img = load_img(root_train)
        self.transform = transform
        self.transform_label = transform_label
        self.train_label_img = label_img_list(load_label_img(root_train_label))
        
    def __len__(self):
        return len(self.train_img)
    
    def __getitem__(self, index):
        img = self.transform(self.train_img[index])
        label = self.transform_label(self.train_label_img[index])
        return img, label

class valset(data.Dataset):
    def __init__(self, transform = None, root_val = None, root_val_label = None, transform_label = None):
        self.val_img = load_img(root_val)
        self.transform = transform
        self.transform_label = transform_label
        self.val_label_img = label_img_list(load_label_img(root_val_label))
        
    def __len__(self):
        return len(self.val_img);
    
    def __getitem__(self, index):
        img = self.transform(self.val_img[index])
        label = self.transform_label(self.val_label_img[index])
        return img, label

    
class testset(data.Dataset):
    def __init__(self, transform = None, root_test = None, root_test_label = None, transform_label = None):
        self.test_img = load_img(root_test)
        self.transform = transform
        self.transform_label = transform_label
        self.test_label_img = label_img_list(load_label_img(root_test_label))
        
    def __len__(self):
        return len(self.test_img)
    
    def __getitem__(self, index):
        img = self.transform(self.test_img[index])
        label = self.transform_label(self.test_label_img[index])
        return img, label
traindataset = trainset(transform_img, '/content/gdrive/My Drive/Camvid dataset/train/', '/content/gdrive/My Drive/Camvid dataset/train_labels/', transform_img_label)
# valdataset = valset(transform_img, '/content/gdrive/My Drive/Camvid dataset/val/', '/content/gdrive/My Drive/Camvid dataset/val_labels/', transform_img_label)
testdataset = testset(transform_img, '/content/gdrive/My Drive/Camvid dataset/test/', '/content/gdrive/My Drive/Camvid dataset/test_labels/', transform_img_label)

Defining the train and test loader with batch size 1, for not filling up the gpu space.

In [0]:
train_loader = data.DataLoader(traindataset, batch_size = 1, shuffle=True,  num_workers=4)
test_loader = data.DataLoader(testdataset, batch_size = 1, shuffle=True,  num_workers=4)

Defining the Unet model, the first part of the U is made by conv2d class and the second part is made by convTranspose2d.

In [0]:
class u_net(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(3, 64, 3)
      self.conv2 = nn.Conv2d(64, 128, 3)
      self.conv3 = nn.Conv2d(128, 256, 3)
      self.conv4 = nn.Conv2d(256, 512, 3)
      self.conv5 = nn.Conv2d(512, 1024, 3)
      self.conv6 = nn.Conv2d(1024, 512, 3)
      self.conv7 = nn.Conv2d(512, 512, 3)
      self.conv8 = nn.Conv2d(512, 256, 3)
      self.conv9 = nn.Conv2d(256, 256, 3)
      self.conv10 = nn.Conv2d(256, 128, 3)
      self.conv11 = nn.Conv2d(128, 128, 3)
      self.conv12 = nn.Conv2d(64, 64, 3)
      self.b1 = nn.BatchNorm2d(64)
      self.b2 = nn.BatchNorm2d(128)
      self.b3 = nn.BatchNorm2d(256)
      self.b4 = nn.BatchNorm2d(512)
      self.b5 = nn.BatchNorm2d(1024)
      self.convT1 = nn.ConvTranspose2d(1024, 512, 2, 2)
      self.convT2 = nn.ConvTranspose2d(512, 256, 2, 2)
      self.convT3 = nn.ConvTranspose2d(256, 256, 2, 2)
      self.convT4 = nn.ConvTranspose2d(128, 64, 2, 2)
      self.convT5 = nn.ConvTranspose2d(64, 32, 2, 2)
      self.pool1 = nn.MaxPool2d(2, 2)

    def forward(self, x):
      x = F.relu(self.b1(self.conv1(x)))
      x = F.relu(self.b1(self.conv12(x)))
      x = F.relu(self.b2(self.conv2(x)))
      x = self.pool1(x)
      x = F.relu(self.b2(self.conv11(x)))
      x = F.relu(self.b3(self.conv3(x)))
      x1 = x
      x1 = x1[:, :, int((58 - 24)/2) : int((58 + 24)/2), int((58 - 24)/2) : int((58 + 24)/2)];
      x = self.pool1(x)
      x = F.relu(self.b3(self.conv9(x)))
      x = F.relu(self.b4(self.conv4(x)))
      x2 = x
      x2 = x2[:, :, int((25 - 16)/2) : int((25 + 16)/2), int((25 - 16)/2) : int((25 + 16)/2)];
      x = self.pool1(x)
      x = F.relu(self.b4(self.conv7(x)))
      x = F.relu(self.b5(self.conv5(x)))
      x = self.b4(self.convT1(x))
      x = torch.cat((x2, x), dim = 1)
      x = F.relu(self.b4(self.conv6(x)))
      x = F.relu(self.b4(self.conv7(x)))
      x = self.b3(self.convT2(x))
      x = torch.cat((x1, x), dim = 1)
      x = F.relu(self.b3(self.conv8(x)))
      x = F.relu(self.b3(self.conv9(x)))
      x = self.b3(self.convT3(x))
      x = F.relu(self.b2(self.conv10(x)))
      x = F.relu(self.b2(self.conv11(x)))
      x = F.relu(self.b2(self.conv11(x)))
      x = F.relu(self.b2(self.conv11(x)))
      x = self.b1(self.convT4(x))
      x = self.convT5(x)
      del x1
      del x2
      return x
net = u_net()
print(net)

u_net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv10): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (b1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
UNET = u_net()
UNET.to(device)


cuda:0


u_net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv10): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (b1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True

Defining the loss as crossentropyloss and the other parameters such as learning rate etc. 

In [0]:

loss1 = nn.CrossEntropyLoss()
optimizer = optim.Adam(UNET.parameters(), lr = 0.0001, betas = (0.9, 0.999), eps = 1e-08, weight_decay=0, amsgrad=False)

Defining the loop for training

In [0]:
for epoch in range(200):
  for i, data in enumerate(train_loader):  
    inputs, labels = data;
    if labels.size() == torch.Size([1, 1, 128, 128]):
      labels = labels.reshape(1, 128, 128)
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = UNET(inputs)
    loss = loss1(outputs, labels)
    loss.backward()
    optimizer.step()


In [0]:
Saving the model to the gdrive.

In [0]:
PATH = '/content/gdrive/My Drive/camvidmodel/saved1.pth'
torch.save(UNET.state_dict(),PATH); # A state_dict is simply a Python dictionary 
# object that maps each layer to its parameter tensor.
